In [1]:
import ray
import random
from ray.air import session, Checkpoint
from ray.air import DatasetConfig
from ray.data import Dataset
from ray.train.torch import TorchTrainer

In [5]:
# our dummy model function
def model(batch):
    return len(batch) * 0.1 * random.uniform(0,1)

def train_loop():
    # By default, bulk loading is used and returns a Dataset object.
    data_shard: Dataset = session.get_dataset_shard("train")
    loss = 0.0
    # Manually iterate over the data 10 times (10 epochs).
    for epoch in range(1, 11):
        # for each epoch iterate over batches
        num_batches = 0
        for batch in data_shard.iter_batches():
            num_batches += 1
            batch_loss = model(batch)
            loss += batch_loss
        loss /= num_batches * 100
        if epoch % 2 == 0:
            print(f"Doing some training on epoch: {epoch} for batches: {num_batches} and loss over batch: {loss:.3f}")
        session.report({"loss": loss, "epoch": epoch}, 
                       checkpoint=Checkpoint.from_dict({"loss": loss, "epoch": epoch}))
    # View the stats for performance debugging.
    print(data_shard.stats())

In [7]:
# Create our TorchTrainer
from ray.air.config import ScalingConfig

train_ds = ray.data.range_tensor(1000)
trainer = TorchTrainer(train_loop,
                       scaling_config= ScalingConfig(num_workers=1),
                       datasets={"train": train_ds},
                      )
result = trainer.fit()

2022-07-21 12:54:49,043	WARNING base_trainer.py:166 -- When passing `datasets` to a Trainer, it is recommended to reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer `scaling_config`. Not doing so can lead to resource contention or hangs. See https://docs.ray.io/en/master/data/key-concepts.html#example-datasets-in-tune for more info.


Trial name,status,loc,iter,total time (s),loss,epoch,_timestamp
TorchTrainer_fa48d_00000,TERMINATED,127.0.0.1:3727,10,2.43943,0.0271053,10,1658433293


(_execute_read_task pid=3268) E0721 12:54:49.180906000 6139457536 chttp2_transport.cc:1111]          Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"
2022-07-21 12:54:50,207	INFO plugin_schema_manager.py:51 -- Loading the default runtime env schemas: ['/Users/jules/git-repos/ray/python/ray/_private/runtime_env/../../runtime_env/schemas/working_dir_schema.json', '/Users/jules/git-repos/ray/python/ray/_private/runtime_env/../../runtime_env/schemas/pip_schema.json'].
(TorchTrainer pid=3727) 2022-07-21 12:54:51,257	WARNING base_trainer.py:166 -- When passing `datasets` to a Trainer, it is recommended to reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer `scaling_config`. Not doing so can lead to resource contention or hangs. See https://docs.ray.io/en/master/data/key-concepts.html#example-datasets-in-tune for more info.
(BaseWorkerMixin pid=3733) 2022-07-21 12:54:52,202	INFO config

Result for TorchTrainer_fa48d_00000:
  _time_this_iter_s: 0.05585002899169922
  _timestamp: 1658433293
  _training_iteration: 1
  date: 2022-07-21_12-54-53
  done: false
  epoch: 1
  experiment_id: f5e20a35ecfc469e8f35629aaadc69c5
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  loss: 0.02155575421004692
  node_ip: 127.0.0.1
  pid: 3727
  should_checkpoint: true
  time_since_restore: 2.135465145111084
  time_this_iter_s: 2.135465145111084
  time_total_s: 2.135465145111084
  timestamp: 1658433293
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: fa48d_00000
  warmup_time: 0.003225088119506836
  
(BaseWorkerMixin pid=3733) Doing some training on epoch: 2 for batches: 20 and loss over batch: 0.023
(BaseWorkerMixin pid=3733) Doing some training on epoch: 4 for batches: 20 and loss over batch: 0.024
(BaseWorkerMixin pid=3733) Doing some training on epoch: 6 for batches: 20 and loss over batch: 0.025
(BaseWorkerMixin pid=3733) Doing some training on epoch: 8 f

2022-07-21 12:54:53,937	INFO tune.py:737 -- Total run time: 4.88 seconds (4.76 seconds for the tuning loop).


In [7]:
print(result.metrics)

{'loss': 0.024684769441396207, 'epoch': 100, '_timestamp': 1657839954, '_time_this_iter_s': 0.035117149353027344, '_training_iteration': 100, 'time_this_iter_s': 0.035944223403930664, 'should_checkpoint': True, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 100, 'trial_id': '7f364_00000', 'experiment_id': '5b6262e8afa148a6a87455dfbbdaad79', 'date': '2022-07-14_16-05-54', 'timestamp': 1657839954, 'time_total_s': 5.424535036087036, 'pid': 1526, 'hostname': 'Juless-MacBook-Pro-16', 'node_ip': '127.0.0.1', 'config': {}, 'time_since_restore': 5.424535036087036, 'timesteps_since_restore': 0, 'iterations_since_restore': 100, 'warmup_time': 0.002769947052001953, 'experiment_tag': '0'}


In [74]:
result.metrics["loss"]

0.23612264564954022

In [75]:
result.checkpoint.to_dict()['loss']

0.23612264564954022